# Proof of inclusion in Bitcoin

This notebook produces the merkle tree of a certain block in bitcoin, produces the proof of inclusion of a transaction and verifies it. 

In [1]:
import hashlib
import codecs
import math

In [2]:
def decode(v):
    '''decode and inverse to little endian'''
    return codecs.decode(v, 'hex')[::-1]

def hash256(s):
    '''two rounds of sha256'''
    return hashlib.sha256(hashlib.sha256(s).digest()).digest()

def hash2(v):
    '''hash two times the value and encode the little endian'''
    h= hash256(v)
    return codecs.encode(h[::-1], 'hex')

## We define the merkle tree class

In [3]:
class MerkleTree:
    def __init__(self,leaves):
        self.levels=[]
        self.leaves=leaves
        self.merkle(leaves)
        self.root=self.levels[-1][0]
        
    '''Hash pairs of items recursively until a single value is obtained'''
    def merkle(self,hashList):
        self.levels.append(hashList)
        hashList=[decode(v) for v in hashList]
        if len(hashList) == 1:
            return hashList[0]
        newHashList = []
        # Process pairs. For odd length, the last is skipped
        newHashList=[hash2(hashList[i]+ hashList[i+1]) for i in range(0, len(hashList)-1, 2)]
        if len(hashList) % 2 == 1: # odd, hash last item twice
            newHashList.append(hash2(hashList[-1]+ hashList[-1]))
        return self.merkle(newHashList)
    
    '''Print the tree in readable format'''
    def __str__(self):
        return "\n".join(["---".join([node[:7].decode() for node in lvl]) for lvl in self.levels[::-1]])


## We also define the block that stores the merkle tree

In [4]:
class Block:
    def __init__(self, txs):
        self.merkleTree=MerkleTree(txs)

## We define the actors: the full node and light node classes
The full node create the blocks with the transactions and the light node checks the validity of the proof of inclusion

In [5]:
'''Full node which creates block and generates the proof of inclusion'''
class FullNode:
    def __init__(self):
        self.blocks=[]
        
    def createBlock(self,txs):
        self.blocks.append(Block(txs))
    
    '''Create the proof of inclusion of a certain transaction specified by the id'''
    def createProof(self,blockIdx, idx, hashList=[], level=0):
        _tree=self.blocks[blockIdx].merkleTree
        if level==len(_tree.levels)-1:
            return hashList
        siblingIdx=idx+1 if idx%2==0 else idx-1
        hashList.append(_tree.levels[level][siblingIdx])
        #print(_hashList)
        return self.createProof(blockIdx, math.floor(idx/2),hashList, level+1)
        

In [6]:
class LightNode:
    '''Returns true if the transaction id is in the tree'''
    def checkInclusion(self, inputToCheck, path, idx, trueRoot):
        if len(path)==0:
            print("root calculated: "+ inputToCheck.decode()+"\n true root:"+trueRoot.decode())
            return inputToCheck==trueRoot
        vs=[inputToCheck, path[0]]
        hashed=hash2(decode(vs[idx%2])+ decode(vs[(idx+1)%2]))
        return self.checkInclusion(hashed, path[1:], math.floor(idx/2), trueRoot)

## Here we verify the bitcoin block  [170 861](https://blockchair.com/bitcoin/block/170861) validating 8 transactions 

In [7]:
txHashes = [
b"338bbd00b893c384eb2b11e70f3875447297c5f20815499e787867df4538e48d",
b"1ad1138c6064dd17d0a4d12016d629c18f15fc9d1472412945f9c91a696689c7",
b"c77834d14d66729014b06fcf45c5f82af4bdd9d816e787f01bfa525cfa147014",
b"bb3d83398d7517fe643b2421d795e73c342b6a478ef53acdaab35dbdffbbcdb5",
b"38d563caf0e9ed103515cab09e40e49da0ccb8c0765ce304f9556e5bc62e8ff5",
b"8fc0507359d0122fa14b5887034d857bd69c8bc0e74c8dd428c2fc098595c285",
b"9db9fe6d011c1c7e997418aeec78ccb659648cfc915b2ff1154cabb41359ac70",
b"3c72fdb7e38e4437faa9e5789df6b51505de014b062361ef47578244d5025628"
]
original_merkle_root=b"acb5aeb11e2a607e610b90f2722cf68aec719af2a2fd6a6af179764e90169af4"

## The full node create the merkle tree with the given transactions ids

In [8]:
fullNode=FullNode()
fullNode.createBlock(txHashes)

In [9]:
print(fullNode.blocks[-1].merkleTree)

acb5aeb
e2d23ad---13f0f97
8e45845---85b994b---e2c81be---9511f44
338bbd0---1ad1138---c77834d---bb3d833---38d563c---8fc0507---9db9fe6---3c72fdb


![merkletree](merkletree.png)

## Check that the merkle root from our tree is the same as in the real bitcoin

In [10]:
print(fullNode.blocks[-1].merkleTree.root==original_merkle_root)

True


## Now as a light client we are interested in checking if the third transaction [c77834d...](https://blockchair.com/bitcoin/transaction/338bbd00b893c384eb2b11e70f3875447297c5f20815499e787867df4538e48d) is indeed in the block because we are the receiver.



In [11]:
idxTransactionToCheck=2

## We ask the full node the proof of inclusion

In [12]:
lightNode=LightNode()
tree=fullNode.blocks[-1].merkleTree
transactionToCheck=tree.levels[0][idxTransactionToCheck]

In [13]:
proofOfInclusion=fullNode.createProof(-1, idxTransactionToCheck, hashList=[])

In [14]:
print([h[:7].decode() for h in proofOfInclusion])

['bb3d833', '8e45845', '13f0f97']


![merkleproof](./merkleproof.png)

## We can now as a light client check the validity of this proof

In [15]:
lightNode.checkInclusion(transactionToCheck,proofOfInclusion, idxTransactionToCheck, fullNode.blocks[-1].merkleTree.root)

root calculated: acb5aeb11e2a607e610b90f2722cf68aec719af2a2fd6a6af179764e90169af4
 true root:acb5aeb11e2a607e610b90f2722cf68aec719af2a2fd6a6af179764e90169af4


True

## The payment is verified!